# Getting set up

## Setting up the notebook

In [ ]:
### Installing dependencies
!pip install openai

!apt-get update
!apt-get install -y iverilog

Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://cli.github.com/packages stable/main amd64 Packages [356 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [83.8 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease [24.6 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [6,388 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [

## Use LLM to generate Verilog code from natural language description

### Example 5: Dice Roller

In [ ]:
!mkdir -p dice_roller
!cd dice_roller && curl -O https://raw.githubusercontent.com/FCHXWH823/LLM4ChipDesign/fe806e8f8b7cb8442ce161f452d070cfcf953656/VerilogGenBenchmark/TestBench/dice_roller_tb.v

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3416  100  3416    0     0  42441      0 --:--:-- --:--:-- --:--:-- 42700


In [ ]:
verilog_generation_prompt = '''
I am trying to create a Verilog model for a simulated dice roller. It must meet the following specifications:
    - Inputs:
        - Clock
        - Active-low reset
        - Die select (2-bits)
        - Roll
    - Outputs:
        - Rolled number (up to 8-bits)

The design should simulate rolling either a 4-sided, 6-sided, 8-sided, or 20-sided die, based on the input die select. It should roll when the roll input goes high and output the random number based on the number of sides of the selected die.

How would I write a design that meets these specifications?
Do not give me the Testbench give me the verilog code only.
Correct the following error : error: port ``rst_n'' is not a port of dut.
1 error(s) during elaboration.
Error: Invalid roll result for 4-sided die:   0
Results for die_select 00:
  Rolled           1:        298 times
  Rolled           2:        234 times
  Rolled           3:        234 times
  Rolled           4:        233 times
Results for die_select 01:
  Rolled           1:        150 times
  Rolled           2:        156 times
  Rolled           3:        178 times
  Rolled           4:        174 times
  Rolled           5:        202 times
  Rolled           6:        140 times
Results for die_select 10:
  Rolled           1:        132 times
  Rolled           2:        112 times
  Rolled           3:        120 times
  Rolled           4:         74 times
  Rolled           5:        166 times
  Rolled           6:        114 times
  Rolled           7:        116 times
  Rolled           8:        166 times
Results for die_select 11:
  Rolled           1:        116 times
  Rolled           2:         48 times
  Rolled           3:         60 times
  Rolled           4:         24 times
  Rolled           5:         36 times
  Rolled           6:         60 times
  Rolled           7:         46 times
  Rolled           8:         60 times
  Rolled           9:         12 times
  Rolled          10:         34 times
  Rolled          11:         24 times
  Rolled          12:         70 times
  Rolled          13:         58 times
  Rolled          14:         34 times
  Rolled          15:         22 times
  Rolled          16:         48 times
  Rolled          17:         70 times
  Rolled          18:         60 times
  Rolled          19:         84 times
  Rolled          20:         34 times
Testbench completed with           1 errors.
'''

In [ ]:
from openai import OpenAI
import os
client = OpenAI(base_url="https://openrouter.ai/api/v1",
    api_key = "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"
)

completion = client.chat.completions.create(
  model = "arcee-ai/trinity-large-preview:free",
  messages=[{"role":"user","content":verilog_generation_prompt}],
  max_tokens=129000, # limits the maximum number of tokens (words or pieces of words) that the model will generate in its response.
  stream=False
)

print(completion.choices[0].message.content)

```verilog
module dice_roller (
    input clk,
    input rst_n,
    input [1:0] die_select,
    input roll,
    output reg [7:0] rolled_number
);

// Define die sides based on die_select
parameter D4 = 2'b00;
parameter D6 = 2'b01;
parameter D8 = 2'b10;
parameter D20 = 2'b11;

reg [7:0] sides;
reg [7:0] counter;
reg rolling;

// Determine number of sides
always @(*) begin
    case (die_select)
        D4:   sides = 8'd4;
        D6:   sides = 8'd6;
        D8:   sides = 8'd8;
        D20:  sides = 8'd20;
        default: sides = 8'd6;
    endcase
end

// Roll detection
always @(posedge clk or negedge rst_n) begin
    if (!rst_n) begin
        rolling <= 1'b0;
    end else if (roll) begin
        rolling <= 1'b1;
    end else if (rolling && counter >= sides) begin
        rolling <= 1'b0;
    end
end

// Counter for generating random number
always @(posedge clk or negedge rst_n) begin
    if (!rst_n) begin
        counter <= 8'd0;
        rolled_number <= 8'd1;
    end else if (rolling) 

In [ ]:
output_verilog_code = '''
module dice_roller (
    input clk,
    input rst_n,
    input [1:0] die_select,
    input roll,
    output reg [7:0] rolled_number
);

// Define die sides based on die_select
parameter D4 = 2'b00;
parameter D6 = 2'b01;
parameter D8 = 2'b10;
parameter D20 = 2'b11;

reg [7:0] sides;
reg [7:0] counter;
reg rolling;

// Determine number of sides
always @(*) begin
    case (die_select)
        D4:   sides = 8'd4;
        D6:   sides = 8'd6;
        D8:   sides = 8'd8;
        D20:  sides = 8'd20;
        default: sides = 8'd6;
    endcase
end

// Roll detection
always @(posedge clk or negedge rst_n) begin
    if (!rst_n) begin
        rolling <= 1'b0;
    end else if (roll) begin
        rolling <= 1'b1;
    end else if (rolling && counter >= sides) begin
        rolling <= 1'b0;
    end
end

// Counter for generating random number
always @(posedge clk or negedge rst_n) begin
    if (!rst_n) begin
        counter <= 8'd0;
        rolled_number <= 8'd1;
    end else if (rolling) begin
        counter <= counter + 8'd1;
        if (counter >= sides) begin
            rolled_number <= (counter % sides) + 8'd1;
        end
    end else begin
        counter <= 8'd0;
    end
end

endmodule
'''
filename = "dice_roller/dice_roller.v"
# Write the extracted Verilog code to the file
with open(filename, "w") as f:
    f.write(output_verilog_code)

In [ ]:
!cd dice_roller/ && iverilog -o dice_roller.vvp dice_roller.v dice_roller_tb.v && vvp dice_roller.vvp

VCD info: dumpfile my_design.vcd opened for output.
Results for die_select 00:
  Rolled           1:       1000 times
Results for die_select 01:
  Rolled           1:       1000 times
Results for die_select 10:
  Rolled           1:       1000 times
Results for die_select 11:
  Rolled           1:       1000 times
Testbench completed successfully.
